<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#VarianceComponentModels.jl" data-toc-modified-id="VarianceComponentModels.jl-1">VarianceComponentModels.jl</a></span><ul class="toc-item"><li><span><a href="#Package-Features" data-toc-modified-id="Package-Features-1.1">Package Features</a></span></li></ul></li><li><span><a href="#Heritability-Analysis" data-toc-modified-id="Heritability-Analysis-2">Heritability Analysis</a></span><ul class="toc-item"><li><span><a href="#Data-files" data-toc-modified-id="Data-files-2.1">Data files</a></span></li><li><span><a href="#Read-in-binary-SNP-data" data-toc-modified-id="Read-in-binary-SNP-data-2.2">Read in binary SNP data</a></span></li><li><span><a href="#EUR_subset" data-toc-modified-id="EUR_subset-2.3"><code>EUR_subset</code></a></span></li><li><span><a href="#Empirical-kinship-matrix" data-toc-modified-id="Empirical-kinship-matrix-2.4">Empirical kinship matrix</a></span></li><li><span><a href="#Simulating-phenotypes" data-toc-modified-id="Simulating-phenotypes-2.5">Simulating phenotypes</a></span></li><li><span><a href="#Phenotypes" data-toc-modified-id="Phenotypes-2.6">Phenotypes</a></span></li><li><span><a href="#Pre-processing-data-for-heritability-analysis" data-toc-modified-id="Pre-processing-data-for-heritability-analysis-2.7">Pre-processing data for heritability analysis</a></span></li><li><span><a href="#Heritability-of-single-trait" data-toc-modified-id="Heritability-of-single-trait-2.8">Heritability of single trait</a></span></li><li><span><a href="#Multivariate-trait-analysis" data-toc-modified-id="Multivariate-trait-analysis-2.9">Multivariate trait analysis</a></span></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-2.10">Exercise</a></span></li></ul></li><li><span><a href="#Testing-SNP-association-using-maximum-likelihoods-of-variance-component-models" data-toc-modified-id="Testing-SNP-association-using-maximum-likelihoods-of-variance-component-models-3">Testing SNP association using maximum likelihoods of variance component models</a></span><ul class="toc-item"><li><span><a href="#Fit-the-null-model" data-toc-modified-id="Fit-the-null-model-3.1">Fit the null model</a></span></li><li><span><a href="#Fit-the-alternative-model" data-toc-modified-id="Fit-the-alternative-model-3.2">Fit the alternative model</a></span></li><li><span><a href="#Likelihood-ratio-test" data-toc-modified-id="Likelihood-ratio-test-3.3">Likelihood ratio test</a></span></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-3.4">Exercise</a></span></li></ul></li></ul></div>

# Heritability analysis and testing SNP association using maximum likelihoods of variance component models 

**ASHG OpenMendel Workshop**

**Juhyun Kim, juhkim111@ucla.edu**

**Department of Biostatistics, UCLA**

**Oct 2020**

Machine information:

In [1]:
versioninfo()

Julia Version 1.5.0
Commit 96786e22cc (2020-08-01 23:44 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i9-9920X CPU @ 3.50GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


## VarianceComponentModels.jl

[`VarianceComponentModels.jl`](https://github.com/OpenMendel/VarianceComponentModels.jl/) is a package that resides in [OpenMendel](https://github.com/OpenMendel) ecosystem. It implements computation routines for fitting and testing variance component model of form 

$$\text{vec}(Y) \sim \text{Normal}(XB, \Sigma_1 \otimes V_1 + \cdots + \Sigma_m \otimes V_m)$$

where $\otimes$ is the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product). 

In this model, **data** is represented by 

* `Y`: `n x d` response matrix 
* `X`: `n x p` covariate matrix 
* `V=(V1,...,Vm)`: a tuple of `m` `n x n` covariance matrices

and **parameters** are

* `B`: `p x d` mean parameter matrix
* `Σ=(Σ1,...,Σm)`: a tuple of `m` `d x d` variance components. 



### Package Features 
* [Heritability analysis](https://openmendel.github.io/VarianceComponentModels.jl/latest/man/heritability/#Heritability-Analysis-1) in genetics
* Maximum likelihood estimation (MLE) and restricted maximum likelihood estimation (REML) of mean parameters $B$ and variance component parameters $Σ$
* Allow constraints in the mean parameters $B$
* Choice of optimization algorithms: [Fisher scoring](https://books.google.com/books?id=QYqeYTftPNwC&lpg=PP1&pg=PA142#v=onepage&q&f=false) and [minorization-maximization algorithm](http://hua-zhou.github.io/media/pdf/ZhouHuZhouLange19VCMM.pdf)

## Heritability Analysis

Variance component estimation can be used to estimate heritability of a quantitative trait. 

### Data files

For this analysis, we use a sample data set [`EUR_subset`](https://openmendel.github.io/SnpArrays.jl/latest/#Example-data-1) from `SnpArrays.jl`. This data set is available in the `data` folder of the package. 



In [5]:
using SnpArrays

In [6]:
datapath = normpath(SnpArrays.datadir())

"/home/juhkim111/.julia/packages/SnpArrays/YOSme/data"

`EUR_subset.bed`, `EUR_subset.bim`, and `EUR_subset.fam` is a set of Plink files in binary format.

In [7]:
using Glob
readdir(glob"EUR_subset.*", datapath)

3-element Array{String,1}:
 "/home/juhkim111/.julia/packages/SnpArrays/YOSme/data/EUR_subset.bed"
 "/home/juhkim111/.julia/packages/SnpArrays/YOSme/data/EUR_subset.bim"
 "/home/juhkim111/.julia/packages/SnpArrays/YOSme/data/EUR_subset.fam"

### Read in binary SNP data 

We use the [`SnpArrays.jl`](https://openmendel.github.io/SnpArrays.jl/latest) package to read in binary SNP data and compute the empirical kinship matrix. 

In [8]:
# read in genotype data from Plink binary file
const EUR_subset = SnpArray(SnpArrays.datadir("EUR_subset.bed"))

379×54051 SnpArray:
 0x03  0x03  0x03  0x02  0x02  0x03  …  0x03  0x03  0x03  0x03  0x03  0x03
 0x03  0x02  0x03  0x02  0x03  0x03     0x03  0x03  0x03  0x03  0x03  0x03
 0x03  0x03  0x03  0x03  0x03  0x03     0x02  0x02  0x02  0x03  0x03  0x02
 0x03  0x03  0x03  0x00  0x03  0x03     0x03  0x03  0x03  0x03  0x03  0x03
 0x03  0x03  0x03  0x00  0x03  0x03     0x02  0x02  0x02  0x03  0x03  0x03
 0x02  0x03  0x03  0x03  0x03  0x03  …  0x03  0x03  0x03  0x03  0x03  0x02
 0x02  0x03  0x03  0x02  0x02  0x03     0x03  0x03  0x02  0x02  0x03  0x03
 0x02  0x03  0x03  0x03  0x02  0x02     0x03  0x03  0x03  0x03  0x03  0x03
 0x03  0x03  0x03  0x00  0x02  0x03     0x03  0x03  0x03  0x03  0x03  0x03
 0x02  0x03  0x03  0x02  0x03  0x02     0x03  0x03  0x03  0x03  0x03  0x03
 0x03  0x03  0x03  0x02  0x03  0x03  …  0x03  0x03  0x02  0x02  0x03  0x03
 0x03  0x03  0x03  0x02  0x03  0x03     0x03  0x03  0x03  0x03  0x03  0x02
 0x03  0x02  0x03  0x02  0x02  0x03     0x03  0x03  0x03  0x03  0x03  0x03
    ⋮

### `EUR_subset` 

`EUR_subset` contains **379** individuals and **54,051** SNPs. There is no missing genotype in `EUR_subset`.

Minor allele frequencies (MAF) for each SNP:

In [9]:
maf_EUR = maf(EUR_subset)

54051-element Array{Float64,1}:
 0.09762532981530347
 0.01319261213720313
 0.04485488126649073
 0.48944591029023743
 0.32189973614775724
 0.09102902374670185
 0.3733509234828496
 0.05277044854881263
 0.0554089709762533
 0.11345646437994727
 0.20448548812664913
 0.16226912928759896
 0.27176781002638517
 ⋮
 0.341688654353562
 0.13192612137203164
 0.24802110817941958
 0.21240105540897103
 0.12532981530343013
 0.13192612137203164
 0.07387862796833777
 0.07783641160949872
 0.13588390501319259
 0.0554089709762533
 0.01319261213720313
 0.02638522427440637

Histogram of minor allele frequency:

In [ ]:
## generate a histogram of MAF
# using Plots, PyPlot
# gr(size=(600, 500), html_output_format=:png)
# hist_maf = histogram(maf_EUR, xlab = "Minor Allele Frequency (MAF)", 
#                    ylab = "Number of SNPs", label="")
# png(hist_maf, "hist_MAF.png")

![image info](./hist_MAF.png)

Note that about 29% of SNPs have their MAF < 0.05. 

In [14]:
count(!iszero, maf_EUR .< 0.05) / length(maf_EUR)

0.2914839688442397

### Empirical kinship matrix

For a measure of relatedness, we compute empirical kinship matrix based on all SNPs by the genetic relation matrix (GRM). If there are missing genotypes, they are imputed on the fly by drawing according to the minor allele frequencies.

Kinship coefficients summarize genetic similarity between pairs of individuals. To estimate kinship coefficient $\Phi_{ij}$ between individuals $i$ and $j$ using GRM:

$$\widehat{\Phi}_{GRMij} = \frac{1}{2S} \sum_{k=1}^S \frac{(G_{ik}-2p_k)(G_{jk}-2p_k)}{2p_k(1-p_k)},$$

where 

* $S$: number of SNPs in this set
* $p_k$: minor allele frequency of SNP $k$
* $G_{ik} \in \{0,1,2\}$: number of copies of minor alleles at the $k$-th SNP of the $i$-th individual

In [11]:
## GRM using SNPs with maf > 0.01 (default) 
Φgrm = grm(EUR_subset; method = :GRM) # classical genetic relationship matrix
# Φgrm = grm(EUR_subset; method = :MoM) # method of moment method
# Φgrm = grm(EUR_subset; method = :Robust) # robust method

379×379 Array{Float64,2}:
  0.526913     -0.010026     -0.0012793    …   0.00536883    0.00713397
 -0.010026      0.500049      0.00147092      -0.00178778   -0.00344277
 -0.0012793     0.00147092    0.521904        -0.0109387    -0.00262695
 -0.00239381    0.00550462    0.00755985      -0.00265867   -0.000141742
 -0.00391296    0.00422806    0.0222034       -0.0107694    -0.00248895
 -0.000555581   0.000696874   0.0125771    …  -0.0100831    -0.00575495
 -0.0095376     0.00231344   -0.00259641      -0.00282701    0.000732385
 -0.00823869    0.00556861    0.0060825       -0.00911662   -0.00638629
  0.00117402   -0.00444907   -0.0029182       -0.00244795    0.00634087
 -0.0111617     0.00436269    0.000537307     -0.00483523   -0.00621726
 -0.00252813   -0.000626719   0.00753937   …  -0.00180836    0.00714953
  0.0112036    -0.0024306     0.00446458      -0.00983116   -0.00296109
 -0.000451414   0.00707358   -0.00620136      -0.00473171   -0.00720874
  ⋮                                 

### Simulating phenotypes 


We simulate phenotype vector from

$$\mathbf{y} \sim \text{Normal}(\mathbf{1}, 0.1 \widehat{\Phi}_{GRM} + 0.9 \mathbf{I})$$

where $\widehat{\Phi}_{GRM}$ is the estimated empirical kinship matrix `Φgrm`. 

The data should be available in `pheno.txt`.

In [ ]:
## simulate `pheno.txt` 
# using LinearAlgebra, DelimitedFiles
# Random.seed!(1234)
# Ω = 0.1 * Φgrm + 0.9 * Matrix(1.0*I, nobs, nobs)
# Ωchol = cholesky(Symmetric(Ω))
# y = ones(nobs) + Ωchol.L * randn(nobs)
# writedlm("pheno.txt", y)

### Phenotypes 

Read in the phenotype data and plot a histogram.

In [15]:
using DelimitedFiles 
pheno = readdlm("pheno.txt")

379×1 Array{Float64,2}:
  1.846582104608307
  0.12019614558345848
  0.5172368025545149
  0.11933401051509984
  1.8407354203053767
  3.155309404417616
  1.518422163488851
  0.737544574135081
  1.4904102203720164
  0.4942945743765427
  0.4566487030521649
  0.9830094325553045
  1.1241872723791884
  ⋮
  0.03800817892237962
  0.7685596964598539
  0.9285816069462199
 -1.3005655794765896
  1.27142883079584
  1.8149274022746835
  2.353663701577899
  1.3085170568729798
  1.2023649250831836
  2.523945778298307
  2.339893360260807
  0.08293644385047372

Histogram of phenotype values:

In [ ]:
## generate histogram of phenotype values
#hist_pheno = histogram(pheno, xlab="Phenotype", ylab="Frequency", label="")
#png(hist_pheno, "hist_pheno.png")

![image info](./hist_pheno.png)

### Pre-processing data for heritability analysis

To prepare variance component model fitting, we form an instance of VarianceComponentVariate. The two covariance matrices are $(2\Phi, I)$.

In [19]:
using VarianceComponentModels, LinearAlgebra
# no. of observations 
nobs = size(pheno, 1)

# form data as VarianceComponentVariate
X = ones(nobs)
EURdata = VarianceComponentVariate(pheno, X, (2Φgrm, Matrix(1.0I, nobs, nobs)))
fieldnames(typeof(EURdata))

┌ Info: Precompiling VarianceComponentModels [813005db-34b4-5f71-be9e-1bbf0a1d8f1c]
└ @ Base loading.jl:1278


(:Y, :X, :V)

In [20]:
EURdata

VarianceComponentVariate{Float64,2,Array{Float64,2},Array{Float64,1},Array{Float64,2}}([1.846582104608307; 0.12019614558345848; … ; 2.339893360260807; 0.08293644385047372], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], ([1.0538262213132035 -0.02005203927901296 … 0.010737654900514978 0.014267936018676418; -0.02005203927901296 1.0000975487266064 … -0.00357555339105328 -0.006885530334701236; … ; 0.010737654900514978 -0.00357555339105328 … 0.9859187797469418 0.026340992959466743; 0.014267936018676418 -0.006885530334701236 … 0.026340992959466743 1.0243852451056221], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]))

### Heritability of single trait 

Before fitting the variance component model, we pre-compute the eigen-decomposition of $2\Phi_{\text{GRM}}$, the rotated responses, and the constant part in log-likelihood, and store them as a TwoVarCompVariateRotate instance, which is re-used in various variane component estimation procedures.

We use Fisher scoring algorithm to fit variance component model for our trait. 

In [21]:
# pre-compute eigen-decomposition 
@time EURdata_rotated = TwoVarCompVariateRotate(EURdata)
fieldnames(typeof(EURdata_rotated))

# form data set for trait 
trait_data = TwoVarCompVariateRotate(EURdata_rotated.Yrot, 
    EURdata_rotated.Xrot, EURdata_rotated.eigval, EURdata_rotated.eigvec, 
    EURdata_rotated.logdetV2)

# initialize model parameters
trait_model = VarianceComponentModel(trait_data)

# estimate variance components
_, _, _, Σcov, = mle_fs!(trait_model, trait_data; solver=:Ipopt, verbose=false)
σ2a = trait_model.Σ[1][1] # additive genetic variance 
σ2e = trait_model.Σ[2][1] # environmental variance 
@show σ2a, σ2e

  0.966252 seconds (2.21 M allocations: 114.370 MiB, 21.52% gc time)

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

(σ2a, σ2e) = (0.5360487493497216, 0.3986105002707737)


(0.5360487493497216, 0.3986105002707737)

Additive genetic variance:

In [22]:
σ2a

0.5360487493497216

Environmental/non-genetic variance:

In [23]:
σ2e

0.3986105002707737

In [24]:
# heritability and its standard error from single trait analysis
h, hse = heritability(trait_model.Σ, Σcov)


[h[1], hse[1]]

2-element Array{Float64,1}:
 0.5735231845909365
 0.2564683288674781

We can also run MM algorithm. 

In [25]:
trait_model = VarianceComponentModel(trait_data)
@time _, _, _, Σcov, = mle_mm!(trait_model, trait_data; verbose=false)
σ2a = trait_model.Σ[1][1]
σ2e = trait_model.Σ[2][1]
@show σ2a, σ2e

  1.142368 seconds (993.74 k allocations: 52.459 MiB, 1.32% gc time)
(σ2a, σ2e) = (0.5242689405328059, 0.4101012322722893)


(0.5242689405328059, 0.4101012322722893)

Heritability and its standard error.

In [26]:
h, hse = heritability(trait_model.Σ, Σcov)
[h[1], hse[1]]

2-element Array{Float64,1}:
 0.5610934036548764
 0.2654835477241166

### Multivariate trait analysis

For the joint analysis of multiple traits, go to [`VarianceComponentModels` documentation](https://openmendel.github.io/VarianceComponentModels.jl/latest/man/heritability/). 

### Exercise

Repeat the above analysis computing the empirical kinship matrix using the method of moment method or the robust method. See [SnpArrays.jl documentation](https://openmendel.github.io/SnpArrays.jl/latest/#Genetic-relationship-matrix-1).

In [27]:
# Φgrm = grm(EUR_subset; method = )

## Testing SNP association using maximum likelihoods of variance component models
credit: [Heritability tutorial by Sarah Ji, Janet Sinsheimer and Hua Zhou](https://github.com/OpenMendel/Tutorials/blob/master/Heritability/HERITABILITY-VCexample.ipynb)

Suppose we want to see a particular SNP has an effect on a given phenotype after accounting for relatedness among individuals. Here we fit variance component model with a single SNP *s* as fixed effect. 

$$\hspace{5em}  \mathbf{y} = \mathbf{X}\mathbf{\beta} + \mathbf{G}_s \gamma + \mathbf{g} + \mathbf{\epsilon} \hspace{5em} (1)$$

\begin{equation}
\begin{array}{ll}
\mathbf{g} \sim N(\mathbf{0}, \sigma_g^2\mathbf{\Phi}) \\
\mathbf{\epsilon} \sim N(\mathbf{0}, \sigma_e^2\mathbf{I})
\end{array}
\end{equation}

where 

* $\mathbf{y}$: phenotype 



and 


* Fixed effects:
    * $\mathbf{X}$: matrix of covariates including intercept
    * $\beta$: vector of covariate effects, including intercept
    * $\mathbf{G}_s$: genotype of SNP *s*
    * $\gamma$: (scalar) association parameter of interest, measuring the effect of genotype on phenotype  
* Random effects:
    * $\mathbf{g}$: random vector of polygenic effects with $\mathbf{g} \sim N(\mathbf{0}, \sigma_g^2 \mathbf{\Phi})$
        * $\sigma_g^2$: additive genetic variance
        * $\mathbf{\Phi}$: matrix of pairwise measures of genetic relatedness 
    * $\epsilon$: random vector with $\epsilon \sim N(\mathbf{0}, \sigma_e^2\mathbf{I})$
        * $\sigma_e^2$: non-genetic variance due to non-genetic effects assumed to be acting independently on individuals



To test whether SNP *s* is associated with phenotype, we fit two models. First consider the model without SNP *s* as fixed effects (aka null model): 

$$\hspace{5em}  \mathbf{y} = \mathbf{X}\mathbf{\beta} + \mathbf{g} + \mathbf{\epsilon} \hspace{5em} (2)$$

and the model with SNP *s* as fixed effects (1). Then we can compare the log likelihood to see if there is improvement in the model fit with inclusion of the SNP of interest. 

### Fit the null model

In [28]:
using VarianceComponentModels
# null data model has two variance components but no SNP fixed effects
# form data as VarianceComponentVariate matrix 
X = ones(nobs)
nulldata = VarianceComponentVariate(pheno, X, (2Φgrm, Matrix(1.0I, nobs, nobs)))

VarianceComponentVariate{Float64,2,Array{Float64,2},Array{Float64,1},Array{Float64,2}}([1.846582104608307; 0.12019614558345848; … ; 2.339893360260807; 0.08293644385047372], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], ([1.0538262213132035 -0.02005203927901296 … 0.010737654900514978 0.014267936018676418; -0.02005203927901296 1.0000975487266064 … -0.00357555339105328 -0.006885530334701236; … ; 0.010737654900514978 -0.00357555339105328 … 0.9859187797469418 0.026340992959466743; 0.014267936018676418 -0.006885530334701236 … 0.026340992959466743 1.0243852451056221], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]))

In [29]:
nullmodel = VarianceComponentModel(nulldata)

VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.0], ([1.0], [1.0]), Array{Float64}(undef,0,1), Char[], Float64[], -Inf, Inf)

In [30]:
@time nulllogl, nullmodel, = fit_mle!(nullmodel, nulldata; algo=:FS)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

(-523.9161903359062, VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.9900140588281151], ([0.5360487493497216], [0.3986105002707737]), Array{Float64}(undef,0,1), Char[], Float64[], -Inf, Inf), ([0.28721820980494434], [0.2789237673497641]), [0.08249430004355703 -0.07781584431646675; -0.07781584431646676 0.07779846799258533], [0.032430583478836504], [0.001051742744777783])

The null model log-likelihood (no SNP effects)

In [31]:
nulllogl

-523.9161903359062

The null model mean effects (a grand mean)

In [32]:
nullmodel.B

1×1 Array{Float64,2}:
 0.9900140588281151

The null model additive genetic variance

In [33]:
nullmodel.Σ[1]

1×1 Array{Float64,2}:
 0.5360487493497216

The null model environmental variance

In [34]:
nullmodel.Σ[2]

1×1 Array{Float64,2}:
 0.3986105002707737

### Fit the alternative model

In [35]:
snp_vec = convert(Vector{Float64}, EUR_subset[:, 10]) 
Xalt = [ones(nobs) snp_vec]
altdata = VarianceComponentVariate(pheno, Xalt, (2Φgrm, Matrix(1.0I, nobs, nobs)))

VarianceComponentVariate{Float64,2,Array{Float64,2},Array{Float64,2},Array{Float64,2}}([1.846582104608307; 0.12019614558345848; … ; 2.339893360260807; 0.08293644385047372], [1.0 3.0; 1.0 3.0; … ; 1.0 2.0; 1.0 3.0], ([1.0538262213132035 -0.02005203927901296 … 0.010737654900514978 0.014267936018676418; -0.02005203927901296 1.0000975487266064 … -0.00357555339105328 -0.006885530334701236; … ; 0.010737654900514978 -0.00357555339105328 … 0.9859187797469418 0.026340992959466743; 0.014267936018676418 -0.006885530334701236 … 0.026340992959466743 1.0243852451056221], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]))

In [36]:
altmodel = VarianceComponentModel(altdata)

VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.0; 0.0], ([1.0], [1.0]), Array{Float64}(undef,0,2), Char[], Float64[], -Inf, Inf)

In [37]:
@time altlogl, altmodel, = fit_mle!(altmodel, altdata; algo=:FS)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

(-523.023009379379, VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.6430890987166976; 0.12582254534184575], ([0.5131884765245557], [0.4165771366329441]), Array{Float64}(undef,0,2), Char[], Float64[], -Inf, Inf), ([0.2881234472862608], [0.280507967380032]), [0.08301512087611872 -0.07854568365271096; -0.07854568365271092 0.0786847197636771], [0.2611312941282091; 0.09394055576183906], [0.06818955277307327 -0.024332309439580764; -0.02433230943958076 0.008824828016843193])

The alternative model log-likelihood:

In [38]:
altlogl

-523.023009379379

The alternative model mean effects (a grand mean)

In [39]:
altmodel.B

2×1 Array{Float64,2}:
 0.6430890987166976
 0.12582254534184575

The alternative model additive genetic variance

In [40]:
altmodel.Σ[1]

1×1 Array{Float64,2}:
 0.5131884765245557

The alternative model environmental variance

In [41]:
altmodel.Σ[2]

1×1 Array{Float64,2}:
 0.4165771366329441

### Likelihood ratio test 

We use likelihood ratio test (LRT) to test the goodness-of-fit between two models. 

Our likelihood ratio test statistic is 1.786 (distributed chi-squared), with one degree of freedom.

In [42]:
using Distributions
LRT = 2(altlogl - nulllogl)

1.7863619130544066

The associated p-value: 

In [43]:
pval = ccdf(Chisq(1), LRT)

0.1813700566785783

We see that adding this SNP as a covariate to the model does not fit significantly better than the null model. In other words, the SNP does not explain more of the variation in our trait.

### Exercise

Use minorization-maximization algorithm (`algo=:MM`) to find MLEs of both null model and alternative model. Then conduct the likelihood ratio test.